In [ ]:
import os
import pandas as pd
from datetime import datetime
import locale

locale.setlocale(locale.LC_TIME,'pt_BR.UTF-8')

pasta = r'D:\PJES_OUTUBRO_24'

# Caminho do log de erros
log_erros = os.path.join(pasta,f"log_erros_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

# Funcao para registra erros no arquivo
def registrar_erro(mensagem):
    with open(log_erros, 'a',encoding='utf-8')as f:
        f.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {mensagem}\n")

# Lista de cargos cota de 300
oficiais = [c.title() for c in ["Oficial da PM", "Tenente", "Capitão", "Major", 
            "Tenente Coronel", "Coronel","Delegado de Policia",
            "Perito Criminal","Medico Legista"]]

#Funcao para calcular verba
def calcular_verba(data_fim, competencia_mes):
    if pd.isna(data_fim):
        return ""
    if data_fim.month == competencia_mes.month and data_fim.year == competencia_mes.year:
        return 223
    elif data_fim < competencia_mes:
        return 423
    else:
        return""

def calcular_valor(row):
    try:
        cargo = str(row['CARGO']).strip().title()
        qtd_cota = float(row['QTDCOTA'])
        if cargo in oficiais:
            return qtd_cota * 300
        else:
            return qtd_cota * 200
    except Exception as e:
        print(f"Erro ao calcular valor para linha: {row}")
        print(e)
        return 0
# Data de referencia da competecia
competencia_data = datetime(2024, 10, 1)

# lista de armazenamento dos dataframes
dfs = []

for arquivo in os.listdir(pasta):
    if arquivo.endswith('.xlsx'):
        caminho_arquivo = os.path.join(pasta, arquivo)
        try:
            try:
                df = pd.read_excel(caminho_arquivo,
                                   sheet_name = 'Planilha de PJES',
                                   skiprows=9,
                                   usecols = "A:L",
                                   engine='openpyxl'
                                   )
            except Exception as e:
                if "MultiCellRange" in str(e):
                    msg_erro = f"Erro ao ler o arquivo '{arquivo}': {str(e)}"
                    print("⚠️", msg_erro)
                    registrar_erro(msg_erro)
                    continue
                else:
                    raise e
                
            
            df.dropna(how='all',inplace=True)
            df.columns = df.columns.str.upper()
            
            #Tratamento
            df['DT TERMINO'] = pd.to_datetime(df['DT TERMINO'], errors='coerce', dayfirst=True)
            df['CARGO'] = df['CARGO'].str.strip().str.title()
            df['QTDCOTA'] = pd.to_numeric(df['QTDCOTA'], errors='coerce').fillna(0)
            df['VALOR'] = df.apply(calcular_valor, axis=1)
            df['EXERCÍCIO'] = 2024
            df['COMPETÊNCIA'] = competencia_data.strftime('%B').upper()
            df['ORIGEM COTA'] = 'DECRETO'
            df['ORIGEM DA INFORMAÇÃO'] = os.path.basename(arquivo).replace('.xlsx','')
            df['OPERATIVA QUE PRESTOU SERVIÇO'] = df['OPERATIVA']
            df['LOCAL DA PRESTAÇÃO DO SERVIÇO'] = df['TITULO']
            df['COTA'] = df['QTDCOTA'].astype(int)
            df['VERBA'] = df['DT TERMINO'].apply(lambda x: calcular_verba(x, competencia_data))
            df['CARGO'] = df['CARGO'].str.strip().str.title()
            df['SITUAÇÃO'] = 'ATIVO'
            df['MATRICULA'] = df['MATRICULA'].astype(str).str.replace(r'\.0$','',regex=True)
            df['NOME'] = df['NOME EXTENSO']
            df['VALOR'] = df['VALOR']

            # Organizar colunas 
            colunas_finais = [
                'EXERCÍCIO', 'COMPETÊNCIA', 'ORIGEM COTA', 'ORIGEM DA INFORMAÇÃO',
                'OPERATIVA QUE PRESTOU SERVIÇO', 'LOCAL DA PRESTAÇÃO DO SERVIÇO',
                'MATRICULA', 'NOME', 'COTA', 'VERBA', 'CARGO', 'SITUAÇÃO','VALOR'
            ]

            
            df_modificado = df[colunas_finais].copy()
            dfs.append(df_modificado)

          
        except Exception as e:
                msg_erro = f"Erro ao processar o arquivo '{arquivo}': {str(e)}"
                print("⚠️", msg_erro)
                registrar_erro(msg_erro)
if dfs:
    data =pd.concat(dfs, ignore_index=True)
    data['COTA'] = pd.to_numeric(data['COTA'], errors='coerce').fillna(0)

    colunas_agrupamento = [
        'EXERCÍCIO', 'COMPETÊNCIA', 'ORIGEM COTA', 'ORIGEM DA INFORMAÇÃO',
        'OPERATIVA QUE PRESTOU SERVIÇO', 'LOCAL DA PRESTAÇÃO DO SERVIÇO',
        'MATRICULA', 'NOME', 'VERBA', 'CARGO', 'SITUAÇÃO'
    ]

    data_agrupado = data.groupby(colunas_agrupamento, as_index=False).agg({
        'COTA':'sum',
        'VALOR': 'sum'
    }).sort_values(by='COTA',ascending=False)

    # Criar tabela dinamica por local
    tabela_pivot = pd.pivot_table(
        data,
        index=['MATRICULA','NOME'],
        columns='LOCAL DA PRESTAÇÃO DO SERVIÇO',
        values='COTA',
        aggfunc='sum',
        fill_value=0
    ).reset_index()

    tabela_pivot.columns.name=None 
    tabela_pivot['TOTAL GERAL']=tabela_pivot.iloc[:,2:].sum(axis=1)
    tabela_pivot = tabela_pivot.sort_values(by='TOTAL GERAL',ascending= False)

    # Criar tabela dinamica por local (apenas VERBA 223)
    tabela_verba_223 = data_agrupado[data_agrupado['VERBA'] == 223]

    tabela_pivot_223 = pd.pivot_table(
        tabela_verba_223,
        index=['MATRICULA','NOME'],
        columns='LOCAL DA PRESTAÇÃO DO SERVIÇO',
        values='COTA',
        aggfunc='sum',
        fill_value=0
    ).reset_index()

    tabela_pivot_223.columns.name=None 
    tabela_pivot_223['TOTAL GERAL']=tabela_pivot_223.iloc[:,2:].sum(axis=1)
    tabela_pivot_223 = tabela_pivot_223.sort_values(by='TOTAL GERAL',ascending= False)

    
    tabela_verba_423 = data_agrupado[data_agrupado['VERBA'] == 423]

    # Tabela total por verba
    tabela_por_verba = data_agrupado.groupby(['MATRICULA','NOME','VERBA'], as_index=False).agg({
        'COTA':'sum',
        'VALOR' :'sum'
    }).sort_values(by=['MATRICULA','VERBA'],ascending=False)

    # Filtra os dados para mostrar apenas as matrículas com Total_Cotas acima de 12
    matriculas_acima_de_12 = tabela_pivot_223[tabela_pivot_223['TOTAL GERAL'] > 12]

    # Salvar em Excel
    agora = datetime.now().strftime("%Y%m%d_%H%M%S")
    nome_saida = f"TABELA_BASE_{agora}.xlsx"
    caminho_saida = os.path.join(pasta, nome_saida)
    nome_aba_mes = competencia_data.strftime('%B').upper()
        
    with pd.ExcelWriter(caminho_saida, engine='openpyxl') as writer:
        data_agrupado.to_excel(writer, index=False, sheet_name=nome_aba_mes) 
        data.to_excel (writer, index=False, sheet_name='DADOS BRUTOS') 
        tabela_pivot.to_excel(writer, index=False, sheet_name='RESUMO POR LOCAL')
        matriculas_acima_de_12.to_excel(writer, index=False, sheet_name='VERBA 223 > 12')
        tabela_verba_423.to_excel(writer, index=False, sheet_name='VERBA 423') 
        

    print(f"\n✅ Arquivo salvo com sucesso em:\n{caminho_saida}")

    # Excluir log se estiver vazio
    if os.path.exists(log_erros) and os.path.getsize(log_erros) == 0:
        os.remove(log_erros)

else:
    print("Nenhum dado processado.")

   